In [21]:
import function as fu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import tensorflow as tf
import random

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [22]:
data = pd.read_csv('data/creditcard.csv')
data.rename(columns= {'LIMIT_BAL':'given_credit',
                       'PAY_0':'status_apr',
                       'PAY_2':'status_may',
                       'PAY_3':'status_jun',
                       'PAY_4':'status_jul',
                       'PAY_5':'status_aug',
                       'PAY_6':'status_sep',

                       'BILL_AMT1':'bill_apr',
                       'BILL_AMT2':'bill_may',
                       'BILL_AMT3':'bill_jun',
                       'BILL_AMT4':'bill_jul',
                       'BILL_AMT5':'bill_aug',
                       'BILL_AMT6':'bill_sep',
                       
                       'PAY_AMT1':'pay_apr',
                       'PAY_AMT2':'pay_may',
                       'PAY_AMT3':'pay_jun',
                       'PAY_AMT4':'pay_jul',
                       'PAY_AMT5':'pay_aug',
                       'PAY_AMT6':'pay_sep',
                       },
                        
            inplace = True)

# As a syntaxis rule, the rest of the column names are changed to lower case
data.columns = data.columns.str.lower()

#Blank spaces are also replaced by "_"
data.columns = data.columns.str.replace(" ", "_")

#Our tatget variable has a very long name, so it is shortened to "default_payment"
data.rename(columns= {'default.payment.next.month':'default_payment'}, inplace = True)

marriage = pd.DataFrame()

#Frequency table by marriage value
marriage["qty_data"] = pd.DataFrame(data.marriage.value_counts())

#Mean age by marriage value:
marriage["mean_age"] = pd.DataFrame(round(data.groupby("marriage")["age"].mean(),2))

#Mean default_payment
marriage["deafult_rate"] = pd.DataFrame(round(data.groupby("marriage")["default_payment"].mean(),2))

data_clean = data
data_clean.loc[(data_clean['marriage']==0),'marriage']=3

education = pd.DataFrame()

#Frequency table by marriage value
education["qty_data"] = pd.DataFrame(data.education.value_counts())

#Mean age by marriage value:
education["mean_age"] = pd.DataFrame(round(data.groupby("education")["age"].mean(),2))

#Mean default_payment
education["deafult_rate"] = pd.DataFrame(round(data.groupby("education")["default_payment"].mean(),2))

data_clean.loc[(data_clean['education']==0) | (data_clean['education']==5) | (data_clean['education']==6),
               'education']=4

status = pd.DataFrame()

#Frequency table by marriage value
status["qty_data"] = pd.DataFrame(data.status_apr.value_counts())

#Mean age by marriage value:
status["mean_age"] = pd.DataFrame(round(data.groupby("status_apr")["age"].mean(),2))

#Mean default_payment
status["deafult_rate"] = pd.DataFrame(round(data.groupby("status_apr")["default_payment"].mean(),2))

data_FE = data_clean
#Set "education" as catigory type
data_FE["education"] = data_FE ["education"].astype("category")

#Creating dummy features
edu_dummies = pd.get_dummies(data_FE['education'], prefix='edu')

#Adding these new dummy variables to our dataset
data_FE = pd.concat( [data_FE, edu_dummies] , axis = 1)
data_FE = data_FE.drop(columns=['education']);

data_FE.columns = data_FE.columns.str.replace(",", "-") # subst. "," for "-"
data_FE.columns = data_FE.columns.str.replace(" ", "")  # deleting spaces
data_FE.columns = data_FE.columns.str.replace("(", "")  # deleting "("
data_FE.columns = data_FE.columns.str.replace(")", "")  # deleting ")"
data_FE.columns = data_FE.columns.str.replace("]", ""); # deleting "]"

data_premod = data_FE
# Separating into input data and target variables

#Eliminating the target column from the input data
X_original =  pd.DataFrame( data_premod.drop(columns='default_payment'))

# Target varibale as independent dataframe
Y_original = pd.DataFrame( data_premod["default_payment"] )

# Standard Scaler from SciKit Learn applies the Z-score normalizaion to the input "X_original"


scaler = StandardScaler()

X_scaled = pd.DataFrame( scaler.fit_transform(X_original) )
X_scaled.columns = X_original.columns;
#X_scaled.iloc[:,1:13].describe()

Y_scaled = Y_original;
data_scaled= pd.concat([X_scaled, Y_scaled], axis=1, join='inner');
    
formatted_data_scaled = pd.DataFrame()
formatted_data_scaled =  pd.melt(data_scaled, ["default_payment"], var_name="feature");

X_train, X_test, y_train, y_test= train_test_split(X_scaled, Y_scaled,test_size=0.2,shuffle = True,random_state = 42)

In [23]:
D_n = np.unique(y_test, return_counts = True)[1][0]
D_p = np.unique(y_test, return_counts = True)[1][1]

In [24]:
D_p * 0.8

1050.4

In [26]:
%%time
p_list = [2,4,6,8,10]
N_list = [[0.2, 0.5], [0.9,1.1], [1.5, 1.8]]
N_list1 = [[0.8,1.2], [0.1,0.8], [0.8, 1.4]]
N_list1 = [ [0.1, 0.8], [0.2,0.5] , [0.6, 1.1]  , [0.7, 1] , [0.8, 1.2], [0.8, 1.4], [0.9,1.4] , [1, 1.1],  [1, 1.2] , [1.1,1.3], [1.5, 1.8] ]
for N1_ratio,N2_ratio in N_list1:
    N1= int(np.unique(y_test, return_counts = True)[1][1] * N1_ratio)
    N2= int(np.unique(y_test, return_counts = True)[1][1] * N2_ratio)
    
    list_con_mat = []
    list_p = []
    list_cost = []
    list_number = []
    list_precision = []
    list_recall = []
    list_f1 = []
    list_auc = []
    list_mcc = []
    list_gmean = []
    
    for p in p_list:
        con_mat,cost, number, precision, recall, f1, auc,mcc, gmean = algorithm_nn(N1 = N1,N2 = N2)
        list_p.append(p)
        list_cost.append(cost)
        list_number.append(number)
        list_precision.append(precision)
        list_recall.append(recall)
        list_f1.append(f1)
        list_con_mat.append(con_mat)
        list_auc.append(auc)
        list_mcc.append(mcc)
        list_gmean.append(gmean)
        
    dict = {'p': list_p, 'num1':list_number, 'con_mat':list_con_mat, 'cost': list_cost, 'list_precision':list_precision, 'list_recall':list_recall, 'list_f1':list_f1, 'list_auc': list_auc, 'list_mcc': list_mcc, 'list_gmean': list_gmean}   

    df = pd.DataFrame(dict) 
    method = 'nn'
    # saving the dataframe 
    df.to_csv('result/case2/result_{}_{}_{}.csv'.format(method,N1,N2))

CPU times: user 31min 32s, sys: 3min 6s, total: 34min 39s
Wall time: 25min 14s


In [25]:
import numpy as np
import math
import pandas as pd
np.set_printoptions(suppress=True)

import keras
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import random
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, roc_auc_score, roc_curve, f1_score, precision_score, recall_score, matthews_corrcoef
import sklearn as sk
from sklearn.preprocessing import StandardScaler
import tempfile
import copy
import warnings 
warnings.filterwarnings('ignore')
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from random import SystemRandom    
# ---------------------------------------------------------------------------


c_10 = 1
c_01 = 3




# -------------------------------------------------------------------------------------
def algorithm(X_train,y_train,X_test,y_test, initial_p, times, print_per_num, total_run, N1, N2):
    

    #initiate
    list_minp = []
    list_mincost = []
    list_num1 = []
    list_p = []
    list_cost = []
    list_opt_cost = []
    list_precision = []
    list_recall = []
    list_f1 = []
    list_point = []
    list_conmat = []
    list_auc = []
    list_mcc = []
    list_gmean = []
    
    num = 0
    

    y_pred0 = Model(1,X_train, y_train, X_test, y_test)
    fpr, fnr, thresholds = fpr_fnr(y_pred0, y_test)
    (interfpr0, interfnr0, threshold,d) = find_cost_intersection(c_01,y_test, fpr, fnr, thresholds)
    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]

    initial_cost= c_01 * interfnr0 * D_p + c_10 * interfpr0 * D_n

    rho = 0.07
    b = 1.6

    for j in range(times):
        p = initial_p
        t= 80
        alpha = 0.9
        C_right = initial_cost
        C_left = initial_cost
        C_feasible = initial_cost
        
        num = num+1


        p_new = p
        p_max = 10
        p_min = 0

        delta = 0
        C_last = 0
        delta_rho = 0

        # -----------------------------

        N =total_run



        #print('{} times'.format(j))
        
        for i in range(1,N+1):
            y_pred = Model(p,X_train, y_train, X_test, y_test)

            # 1
            fpr, fnr, thresholds = fpr_fnr(y_pred, y_test)
            (interfpr, interfnr, threshold,d) = find_cost_intersection(c_01,y_test, fpr, fnr, thresholds)
            (interfpr1,interfnr1,threshold1), (interfpr2,interfnr2,threshold2)= find_limit_point(fpr, fnr, thresholds,N1,N2)
            number_of_1 = D_n * interfpr + D_p * (1-interfnr)

            # 2
            # left
            if interfpr < interfpr1:

                case = 'left'
                interfpr = interfpr1
                interfnr = interfnr1

                C= int(c_01 * interfnr * D_p + c_10 * interfpr * D_n)
                C_left = min(C, C_left)


            # right
            elif interfpr2 < interfpr:

                case = 'right'
                interfpr = interfpr2
                interfnr = interfnr2

                C= int(c_01 * interfnr * D_p + c_10 * interfpr * D_n)
                C_right = min(C, C_right)


            # center
            elif interfpr1<= interfpr <= interfpr2:
                interfpr, interfnr,thresholds = find_cost_intersection_within(c_01,y_test, fpr, fnr, thresholds,N1,N2)
                C= int(c_01 * interfnr * D_p + c_10 * interfpr * D_n)
                C_feasible = min(C_feasible, C)
                case = 'center'


            # 3
            t = alpha * t

            # 4
            C_opt = min(C_left, C_right, C_feasible)

            # 5        
            delta_C = C - C_last 

            # 6
            probability = (C_right + 0.5 * C_feasible) / (C_right + C_left + C_feasible) 

            # 9
            probability2 = np.exp(-delta_C / t)
            if delta_C <= 0 or ( delta_C > 0 and  probability2 > SystemRandom().random() ):

                # 7

                delta_rho = (1 - rho  ** ( (1 - (i/N) ) ** b ) )
                if probability <= SystemRandom().random():
                    delta = (p - p_min) *  delta_rho  # 8
                    p_new = p - delta
                else:
                    delta = (p_max - p) * delta_rho   # 8
                    p_new = p + delta


            C_last = C

            list_p.append(p)
            list_cost.append(C)
            p = p_new
        
        min_cost = min(list_cost)
        min_p = list_p[list_cost.index(min_cost)]
        y_pred = Model(min_p,X_train, y_train, X_test, y_test)
        fpr, fnr, thresholds = fpr_fnr(y_pred, y_test)
        number_of_1,precision,recall,f1, auc,mcc, gmean, con_mat,interfpr,interfnr,cost,opt_cost = metrics(min_p,fpr, fnr, thresholds,N1,N2,X_train, y_train, X_test, y_test)
        
        
        min_p = np.round(min_p,5)
        list_minp.append(min_p)
        list_mincost.append(cost)
        list_opt_cost.append(opt_cost)
        list_num1.append(number_of_1)

        list_precision.append(precision)
        list_recall.append(recall)
        list_f1.append(f1)
        list_auc.append(auc)
        list_mcc.append(mcc)
        list_gmean.append(gmean)
        
        list_point.append((interfpr, interfnr))
        list_conmat.append(con_mat)
        
        
    
    dict = {'p': list_minp, 'cost': list_mincost, 'num1': list_num1, 'list_conmat': list_conmat, 'list_point': list_point, 'list_precision':list_precision, 'list_recall':list_recall, 'list_f1':list_f1, 'list_auc': list_auc, 'list_mcc': list_mcc, 'list_gmean': list_gmean, 'list_opt_cost':list_opt_cost}
    
    df = pd.DataFrame(dict) 
    method = 'ondemand'
    # saving the dataframe 
    df.to_csv('result/case1/result_{}_{}_{}_{}.csv'.format(method,N1,N2,initial_p,))
    
    
# -------------------------------------------------------------------------------------

def metrics(p, fpr, fnr, thresholds,N1,N2,X_train, y_train, X_test, y_test):
    y_pred = Model(p,X_train, y_train, X_test, y_test)
    
    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]
    # 1
    (interfpr, interfnr, threshold,d) = find_cost_intersection(c_01, y_test, fpr, fnr, thresholds)
    (interfpr1,interfnr1,threshold1), (interfpr2,interfnr2,threshold2)= find_limit_point(fpr, fnr, thresholds,N1,N2)
    
    interfpr, interfnr, threshold = find_cost_intersection_within(c_01,y_test, fpr, fnr, thresholds,N1,N2)
    
    y_pred_copy = copy.copy(y_pred)
    y_result = np.where(y_pred_copy>threshold,1,0)
    
    number_of_1 = D_n * interfpr + D_p * (1-interfnr)
    number_of_1 = round(number_of_1)
    
    y_result = np.where(y_pred >= threshold, 1, 0)
    con_mat = confusion_matrix(y_test, y_result).T.flatten().tolist()
    TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
    
    cost = int(c_01 * interfnr * D_p + c_10 * interfpr * D_n)
    optimal_cost = round(c_01 * D_p *d)
    

    if N2 < number_of_1:
        count = 0
        ticket = False
        sum1 = sum(np.where(y_pred>threshold,1,0))
        target = N2 - sum1
        y_pred_copy = copy.copy(y_pred)

        for j in range(len(y_pred)):
            if y_pred[j] == threshold:
                y_pred_copy[j] = 1
                count +=1

            if count == target:
                ticket = True
                break

        if ticket == True:
            y_result = np.where(y_pred_copy >threshold,1,0)
            con_mat = confusion_matrix(y_test, y_result).T.flatten().tolist()
            TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
            number_of_1 = FP + TP

    if N2 < number_of_1 and N1 < D_p and D_p < N2: #50,150
        count = 0
        ticket = False
        sum1 = sum(np.where(y_pred>threshold,1,0))
        target = N2 - sum1
        y_pred_copy = copy.copy(y_pred)
        cost_list = []
        con_mat_list = []

        for j in range(len(y_pred)):
            if y_pred[j] == threshold:
                y_pred_copy[j] = 1
                count +=1
                y_result = np.where(y_pred_copy >threshold,1,0)
                TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
                if FP + TP >N1:
                    cost = FP * c_10 + FN * c_01
                    con_mat_list.append(confusion_matrix(y_test, y_result).T.flatten().tolist())
                    cost_list.append(cost)

            if count == target:
                ticket = True
            break

        if ticket == True:
            y_result = np.where(y_pred >threshold,1,0)
            con_mat = con_mat_list[cost_list.index(min(cost_list))]
            cost = min(cost_list)
            TN, FN, FP, TP = con_mat
            number_of_1 = FP + TP    
        
    p = round(precision_score(y_test, y_result),3)
    r = round(recall_score(y_test, y_result),3)
    f = round(f1_score(y_test, y_result),3)
    auc = 1- round(roc_auc_score(y_test, y_result),3)
    mcc = round(matthews_corrcoef(y_test, y_result),3)
    
    tpr = TP / (TP+FN)
    tnr = TN / (TN+FP)
    gmean = np.sqrt(tpr * tnr)
    
    
    return number_of_1, p,r,f, auc,mcc, gmean, con_mat, interfpr,interfnr,cost,optimal_cost
# --------------------------------------------------------------------------------------------------------------------------
def find_limit_point(fpr, fnr, thresholds,N1,N2):
    (interfpr1,interfnr1,threshold1) = (fpr[N1-1], fnr[N1-1], thresholds[N1-1])
    (interfpr2,interfnr2,threshold2) = (fpr[N2-1], fnr[N2-1], thresholds[N2-1])
    
    return (interfpr1,interfnr1,threshold1), (interfpr2,interfnr2,threshold2)

# -----------------------------------------------------------------------------

def find_cost_intersection(c_01,y_test, fpr, fnr, thresholds):
    list_distance = []
    norm = np.linalg.norm
    
    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]

    k =0.000001
    for j in range(len(fpr)):
        p1 = np.array([0,k])
        p2 = np.array([(D_p/D_n)*c_01*k, 0])

        p3 = np.array([fpr[j], fnr[j]])

        distance = np.abs(norm(np.cross(p2-p1, p1-p3)))/norm(p2-p1)
        list_distance.append(distance)
    index = list_distance.index(min(list_distance))

    return fpr[index], fnr[index], thresholds[index],min(list_distance)
# ---------------------------------------------------------------------------------------------------------------

def find_cost_intersection_within(c_01,y_test, fpr, fnr, thresholds,N1,N2):
    
    (interfpr1,interfnr1,threshold1) = (fpr[N1-1], fnr[N1-1], thresholds[N1-1])
    (interfpr2,interfnr2,threshold2) = (fpr[N2-1], fnr[N2-1], thresholds[N2-1])

    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]
    
    list_distance = []
    norm = np.linalg.norm

    k =0.00001
    for j in range(N1-1, N2):
        p1 = np.array([0,k])
        p2 = np.array([(D_p/D_n)*c_01*k, 0])

        p3 = np.array([fpr[j], fnr[j]])

        distance = np.abs(norm(np.cross(p2-p1, p1-p3)))/norm(p2-p1)
        list_distance.append(distance)

    index = list_distance.index(min(list_distance))
    
    return fpr[N1-1 + index], fnr[N1-1 + index], thresholds[N1-1 + index]

# ---------------------------------------------------------------------------------------------------------------


def algorithm_nn(N1,N2):


        
        
    y_pred = Model(1,X_train, y_train, X_test, y_test)
    fpr, fnr, thresholds = fpr_fnr(y_pred, y_test)
    
    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]
    
    if D_p <= N1:
        num = N1
    elif N2 <= D_p:
        num = N2
    elif N1 < D_p < N2:
        num = D_p
        
    th = sorted(y_pred, reverse = True)[num]
    y_result = np.where(y_pred>th, 1, 0)
    cost = confusion_matrix(y_test, y_result).T[0][1] * c_01 + confusion_matrix(y_test, y_result).T[1][0] * c_10

 
    number = confusion_matrix(y_test, y_result).T[1][1] + confusion_matrix(y_test, y_result).T[1][0]       
    p = round(precision_score(y_test, y_result),3)
    r = round(recall_score(y_test, y_result),3)
    f = round(f1_score(y_test, y_result),3)
    auc = round(roc_auc_score(y_test, y_result),3)
    mcc = round(matthews_corrcoef(y_test, y_result),3)
    
    TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()

    tpr = TP / (TP+FN)
    tnr = TN / (TN+FP)
    gmean = np.sqrt(tpr * tnr)
    
    cost = confusion_matrix(y_test, y_result).T[0][1] * c_01 + confusion_matrix(y_test, y_result).T[1][0] * c_10 
    con_mat = confusion_matrix(y_test, y_result).T.flatten().tolist()
    
    return con_mat,cost, number, p, r, f, auc,mcc, gmean
    



#------------------------------------------------------------------------------------------------------------------------

def algorithm_cs(p, N1,N2):

        
        
    y_pred = Model(p,X_train, y_train, X_test, y_test)
    fpr, fnr, thresholds = fpr_fnr(y_pred, y_test)

    
    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]
    
    if D_p <= N1:
        num = N1
    elif N2 <= D_p:
        num = N2
    elif N1 < D_p < N2:
        num = D_p
    
    th = sorted(y_pred, reverse = True)[num]
    y_result = np.where(y_pred>th, 1, 0)
    TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
    cost = confusion_matrix(y_test, y_result).T[0][1] * c_01 + confusion_matrix(y_test, y_result).T[1][0] * c_10

 
    number = confusion_matrix(y_test, y_result).T[1][1] + confusion_matrix(y_test, y_result).T[1][0]       
    p = round(precision_score(y_test, y_result),3)
    r = round(recall_score(y_test, y_result),3)
    f = round(f1_score(y_test, y_result),3)
    auc = round(roc_auc_score(y_test, y_result),3)
    mcc = round(matthews_corrcoef(y_test, y_result),3)
    
    tpr = TP / (TP+FN)
    tnr = TN / (TN+FP)
    gmean = np.sqrt(tpr * tnr)
    
    
    cost = confusion_matrix(y_test, y_result).T[0][1] * c_01 + confusion_matrix(y_test, y_result).T[1][0] * c_10 
    con_mat = confusion_matrix(y_test, y_result).T.flatten().tolist()
    
    return con_mat,cost, number, p, r, f, auc,mcc, gmean
# ---------------------------------------------------------------------------------------------------------------



def fpr_fnr(y_pred, y_test):
    
    list_fpr = []
    list_fnr = []
    list_th = []
    y_pred_sort=sorted(y_pred, reverse=True)

    for i in range(1,len(y_pred)+1):
        th = y_pred_sort[i-1][0]
        y_result = np.where(y_pred >= th,1,0)
        tn, fn, fp, tp = confusion_matrix(y_test, y_result).T.ravel()
        fpr = fp / (tn+fp)
        fnr = fn / (tp+fn)

        list_fpr.append(fpr)
        list_fnr.append(fnr)
        list_th.append(th)
        
    return [list_fpr, list_fnr, list_th]

# ------------------------------------------------------------------------------------------------------------------------


# define model

def mid_data():
    np.random.seed(0)
    X =  np.random.rand(20000,12) * 20

    y = []
    for p in range(len(X)):


        a = X[p][0] 
        b = X[p][1] 
        c = X[p][2] 
        d = X[p][3] 
        e = X[p][4] 
        f = X[p][5] 
        
        g = math.cos(X[p][6]) * 10
        h = math.cos(X[p][7]) * 10
        i = math.cos(X[p][8]) * 10
        j = math.sin(X[p][9]) * 10
        k = math.sin(X[p][10]) * 10
        l = math.sin(X[p][11]) * 10

        sum_ = a-b+c-d+e-f+g-h+i-j+k-l
        result = 1 if sum_ > 0 else 0

        y.append(result)

    df =  pd.DataFrame(X)
    df['y'] = y

    df_0 = df[df['y'] == 0][:9000]
    df_1 = df[df['y'] == 1][:1000]

    df__0 = df_0[:7200].append(df_1[:800]).sample(frac = 1).reset_index(drop = True)
    df__1 = df_0[7200:].append(df_1[800:]).sample(frac = 1).reset_index(drop = True)

    X_train = df__0.iloc[:, [0,1,2,3,4,5,6,7,8,9]].to_numpy()
    y_train = df__0.iloc[:, [-1]].to_numpy()
    X_test = df__1.iloc[:, [0,1,2,3,4,5,6,7,8,9]].to_numpy()
    y_test = df__1.iloc[:, [-1]].to_numpy()

    return X_train, y_train, X_test, y_test

# ------------------------------------------------------------------------------------------------------------------------
def easy_data():
    np.random.seed(0)
    X =  np.random.rand(20000,12) * 20

    y = []
    for p in range(len(X)):


        a = X[p][0] 
        b = X[p][1] 
        c = X[p][2] 
        d = X[p][3] 
        e = X[p][4] 
        f = X[p][5] 
        
        g = X[p][6] 
        h = X[p][7] 
        i = X[p][8] 
        j = X[p][9] 
        k = X[p][10] 
        l = X[p][11]
        

        sum_ = a-b+c-d+e-f+g-h+i-j+k-l
        result = 1 if sum_ > 0 else 0

        y.append(result)

    df =  pd.DataFrame(X)
    df['y'] = y

    df_0 = df[df['y'] == 0][:9000]
    df_1 = df[df['y'] == 1][:1000]

    df__0 = df_0[:7200].append(df_1[:800]).sample(frac = 1).reset_index(drop = True)
    df__1 = df_0[7200:].append(df_1[800:]).sample(frac = 1).reset_index(drop = True)

    X_train = df__0.iloc[:, [0,1,2,3,4,5,6,7,8,9]].to_numpy()
    y_train = df__0.iloc[:, [-1]].to_numpy()
    X_test = df__1.iloc[:, [0,1,2,3,4,5,6,7,8,9]].to_numpy()
    y_test = df__1.iloc[:, [-1]].to_numpy()

    return X_train, y_train, X_test, y_test

# ------------------------------------------------------------------------------------------------------------------------

def hard_data():
    np.random.seed(0)
    X =  np.random.rand(20000,12) * 20

    y = []
    for p in range(len(X)):


        a = math.cos(X[p][0]) * 10
        b = math.cos(X[p][1]) * 10
        c = math.cos(X[p][2]) * 10
        d = math.sin(X[p][3]) * 10
        e = math.sin(X[p][4]) * 10
        f = math.sin(X[p][5]) * 10 
        
        g = math.cos(X[p][6]) * 10
        h = math.cos(X[p][7]) * 10
        i = math.cos(X[p][8]) * 10
        j = math.sin(X[p][9]) * 10
        k = math.sin(X[p][10]) * 10
        l = math.sin(X[p][11]) * 10

        sum_ = a-b+c-d+e-f+g-h+i-j+k-l
        result = 1 if sum_ > 0 else 0

        y.append(result)

    df =  pd.DataFrame(X)
    df['y'] = y

    df_0 = df[df['y'] == 0][:9000]
    df_1 = df[df['y'] == 1][:1000]

    df__0 = df_0[:7200].append(df_1[:800]).sample(frac = 1).reset_index(drop = True)
    df__1 = df_0[7200:].append(df_1[800:]).sample(frac = 1).reset_index(drop = True)

    X_train = df__0.iloc[:, [0,1,2,3,4,5,6,7,8,9]].to_numpy()
    y_train = df__0.iloc[:, [-1]].to_numpy()
    X_test = df__1.iloc[:, [0,1,2,3,4,5,6,7,8,9]].to_numpy()
    y_test = df__1.iloc[:, [-1]].to_numpy()

    return X_train, y_train, X_test, y_test

# -------------------------------------------------------------------------------------  
    
def reset_random_seeds():
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    random.seed(1)

# -------------------------------------------------------------------------------------

def Model(p, X_train,y_train,X_test,y_test):
    reset_random_seeds()

    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[-1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the Model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    model.fit(X_train,y_train, epochs=30, shuffle = True, batch_size=32 ,class_weight =  {0: 1, 1: p}, verbose =0)
    y_pred = model.predict(X_test, batch_size=32, verbose =0)
    return y_pred



# -------------------------------------------------------------------------------------

def rough_plt(N1,N2):

    list_p = []
    list_cost = []
    list_cost_ = []
    list_inter = []
    list_opt = []
    list_con1 = []
    list_con2 = []

    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]

    for p in range(1,10,1):
        y_pred = Model(p,X_train, y_train, X_test, y_test)
        print(p)


        (interfpr0, interfnr0, thresholds) = find_cost_intersection(c_01,y_test,fpr, fnr, thresholds)
        (interfpr1,interfnr1,threshold1), (interfpr2,interfnr2,threshold2)= find_limit_point(fpr, fnr, thresholds,N1,N2)

        list_opt.append([interfpr, interfnr])
        list_con1.append([interfpr1, interfnr1])
        list_con2.append([interfpr2, interfnr2])

        cost1 = c_01 * interfnr * D_p + c_10 * interfpr * D_n

        if interfpr1 <= interfpr <= interfpr2:
            cost = c_01 * interfnr * D_p + c_10 * interfpr * D_n
            list_inter.append([interfpr,interfnr])

        elif interfpr < interfpr1:
            cost = c_01 * interfnr1 * D_p + c_10 * interfpr1 * D_n
            list_inter.append([interfpr1,interfnr1])

        elif interfpr2 < interfpr:
            cost = c_01 * interfnr2 * D_p + c_10 * interfpr2 * D_n
            list_inter.append([interfpr2,interfnr2])

        list_cost.append(round(cost1))
        list_cost_.append(round(cost))
        list_p.append(p)

    list_rank_p = []
    list_rank_cost = []
    for i in range(5):
        rank = np.where(np.array(list_cost_) == sorted(list_cost_)[i])[0][0]

        rank_p = list_p[rank]
        rank_cost = list_cost_[rank]

        list_rank_p.append(rank_p)
        list_rank_cost.append(rank_cost)


    plt.figure(figsize=(8, 6))
    plt.plot(list_p,list_cost, marker='o', label='optimal')
    plt.plot(list_p,list_cost_, marker='o', label='constraint')
    plt.legend()
    plt.title(f'N1:{N1}, N2:{N2}')
    plt.xlabel(f'Best:{list_rank_p[:10]} \n Best:{list_rank_cost[:10]}')
    plt.savefig(f'rough {N1}, {N2}')
    
    return None
# -------------------------------------------------------------------------------------   
    
def precise_plt(p_start, p_end,N1,N2):


    list_p = []
    list_cost = []
    list_cost_ = []
    list_inter = []
    list_opt = []
    list_con1 = []
    list_con2 = []

    for p in range(p_start,p_end,1):
        p = p/100

        y_pred = Model(p,X_train, y_train, X_test, y_test)
        D_n = np.unique(y_test, return_counts = True)[1][0]
        D_p = np.unique(y_test, return_counts = True)[1][1]


        (interfpr0, interfnr0, thresholds) = find_cost_intersection(c_01,y_test, fpr, fnr, thresholds)
        (interfpr1,interfnr1,threshold1), (interfpr2,interfnr2,threshold2)= find_limit_point(fpr, fnr, thresholds,N1,N2)

        list_opt.append([interfpr, interfnr])
        list_con1.append([interfpr1, interfnr1])
        list_con2.append([interfpr2, interfnr2])

        cost1 = c_01 * interfnr * D_p + c_10 * interfpr * D_n

        if interfpr1 <= interfpr <= interfpr2:
            cost = c_01 * interfnr * D_p + c_10 * interfpr * D_n  
            list_inter.append([interfpr,interfnr])

        elif interfpr < interfpr1:
            cost = c_01 * interfnr1 * D_p + c_10 * interfpr1 * D_n
            list_inter.append([interfpr1,interfnr1])

        elif interfpr2 < interfpr:
            cost = c_01 * interfnr2 * D_p + c_10 * interfpr2 * D_n
            list_inter.append([interfpr2,interfnr2])

        list_cost.append(round(cost1))
        list_cost_.append(round(cost))
        list_p.append(p)

    list_rank_p = []
    list_rank_cost = []
    for i in range(10):
        rank = list_cost_.index([*set(sorted(list_cost_))][i])

        rank_p = list_p[rank]
        rank_cost = list_cost_[rank]

        list_rank_p.append(rank_p)
        list_rank_cost.append(rank_cost)

    plt.figure(figsize=(8, 6))
    plt.scatter(list_p,list_cost, marker='o', label='optimal')
    plt.scatter(list_p,list_cost_, marker='o', label='constraint')
    plt.legend()
    plt.title(f'N1:{N1}, N2:{N2}')
    plt.xlabel(f'Best:{list_rank_p[:10]} \n Best:{list_rank_cost[:10]}')
    plt.savefig(f'precise {N1}, {N2}')
    
    return None


# -------------------------------------------------------------------------------------------